In [1]:
import yaml
import pathlib
import time
from Agent.xwAPI import xwBackend
from utils.construct_prompt import get_api_doc

In [2]:
config_file = "./config/config.yaml"
with open(config_file, mode='r') as file:
    config = yaml.load(file, Loader=yaml.Loader)

agent_config = config["Agent"]
prompt_format = agent_config['ChatGPT_1']['prompt_format']
with open(agent_config['api_doc_path']) as f:
    api_doc = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
if agent_config['API_backend'] == 'xw':
    xw_backend = xwBackend(agent_config['APP_backend'], api_doc)

In [4]:
path_generator = pathlib.Path(f"../output_dir/refined_responses/").glob("**/*")
response_log_paths = [str(path).split("\\")[-1] for path in path_generator if path.is_file()]
excel_file_names = [f"{log_file[2:-5]}.xlsx" for log_file in response_log_paths]
excel_file_names

['DemographicProfile.xlsx', 'Dragging.xlsx', 'DemographicProfile.xlsx']

In [5]:
index = 1
file = f"../dataset/task_sheets/{excel_file_names[index].replace('yaml', 'xlsx')}"
file

'../dataset/task_sheets/Dragging.xlsx'

In [6]:
if file is not None:
    time.sleep(0.5)
    xw_backend.OpenWorkbook(file)

In [7]:
sheet_state = xw_backend.GetSheetsState()
sheet_state

'Sheet state: Sheet "Sheet1" has 2 columns (Headers are A: "Hanging Mass (m2) (kg)", B: "Acceleration (m/s^2)") and 122 rows (1 header row and 121 data rows).'

In [8]:
api_list, api_usage, api_detail_doc = get_api_doc(prompt_format, api_doc)

In [9]:
response_log_path = f"../output_dir/refined_responses/{response_log_paths[index]}"
with open(response_log_path) as file:
    response_log = yaml.load(file, Loader=yaml.Loader)

In [10]:
prompt = """Summarize the each sub-step of refined responses into explanations in natural language:
{refined_response}
Here is the supplementary doc you can reference:
{documentation}
Here is the corresponding sheet state:
{sheet_state}
"""
prompt

'Summarize the each sub-step of refined responses into explanations in natural language:\n{refined_response}\nHere is the supplementary doc you can reference:\n{documentation}\nHere is the corresponding sheet state:\n{sheet_state}\n'

In [11]:
unique_function_names = set([sub_instruction[:sub_instruction.find("(")] for sub_instruction in response_log["refined_response"]])
unique_function_names

{'AutoFill', 'CreateChart', 'CreateSheet', 'SetChartAxis', 'SetChartTitle'}

In [12]:
documentation_for_functions = {"documentation": [api_detail_doc[name] for name in unique_function_names]}

In [13]:
print(prompt.format(refined_response=yaml.dump(response_log),documentation=yaml.dump(documentation_for_functions), sheet_state=sheet_state))

Summarize the each sub-step of refined responses into explanations in natural language:
refined_response:
- AutoFill(source="Sheet1!B2", destination="Sheet1!B2:B122")
- CreateSheet(sheetName="ScatterChart")
- CreateChart(source="Sheet1!A1:B122", destSheet="ScatterChart", chartType="XYScatter",
  chartName="Acceleration vs Hanging Mass", XField=1, YField=[2])
- SetChartTitle(chartName="Acceleration vs Hanging Mass", title="Acceleration vs Hanging
  Mass")
- SetChartAxis(chartName="Acceleration vs Hanging Mass", axis="x", title="Hanging
  Mass (m2) (kg)")
- SetChartAxis(chartName="Acceleration vs Hanging Mass", axis="y", title="Acceleration
  (m/s^2)")

Here is the supplementary doc you can reference:
documentation:
- 'AutoFill(source: str, destination: str)

  Args explanation:

  source (string): The range to auto fill from.

  destination (string): The range to auto fill into.

  # The destination must include the source range.


  Usage example:

  # Example 1: Autofill down columns
